In [121]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [122]:
ratings = pd.read_csv('/Users/sevgicosan/Desktop/Movie Recommendation System /ratings.csv_back')
movies = pd.read_csv('/Users/sevgicosan/Desktop/Movie Recommendation System /movies.csv_back')
ratings_copy = ratings.copy()
movies_copy = movies.copy()

In [172]:
tags = pd.read_csv('/Users/sevgicosan/Desktop/Movie Recommendation System /tags.csv')
tags.columns


Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')

In [123]:
ratings_copy.drop("timestamp", axis = 1, inplace = True)

In [170]:
#ratings.head()
#movies.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [125]:
missing_values_count_ratings = ratings.isnull().sum()
missing_values_count_movies = movies.isnull().sum()

In [126]:
# missing_values_count_ratings
# missing_values_count_movies

In [127]:
top_movie_ids = ratings_copy[ratings_copy['movieId'].isin(ratings_copy['movieId'].value_counts()[ratings_copy['movieId'].value_counts() >= 100].index)].movieId.unique()
top_movie_ratings = ratings.loc[ratings["movieId"].isin(top_movie_ids)]
top_movies = movies.loc[movies["movieId"].isin(top_movie_ids)]

In [161]:
top_movie_ids

array([   296,    306,    307, ..., 127206,   3942, 118202])

In [129]:
top_movie_ratings.shape

(24443380, 4)

In [130]:
movies_and_ratings = pd.merge(top_movies,top_movie_ratings, on = 'movieId')
#movies_and_ratings = movies_and_ratings.drop('timestamp', axis = 1)

In [131]:
movies_and_ratings.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0


In [132]:
movies_and_features = movies_and_ratings.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
#movies_and_features = movies_and_features.fillna(0)


In [144]:
movies_and_features.head(100)

userId,1,2,3,4,5,6,7,8,9,10,...,162532,162533,162534,162535,162536,162537,162538,162539,162540,162541
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,4.0,3.0,4.0,0.0,0.0,4.0,0.0,3.5,...,0.0,4.5,4.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,...,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
mat_movies_and_features = csr_matrix(movies_and_features.values)

In [135]:
mat_movies_and_features.shape

(10326, 162540)

In [145]:
neigh = NearestNeighbors(n_neighbors = 6)
neigh.fit(mat_movies_and_features)

NearestNeighbors()

In [162]:
movie_id = 127206
movie = movies_and_features.loc[movie_id, :]
scores, indices = neigh.kneighbors([movie])

In [163]:
scores

array([[ 0.        , 41.16734142, 41.41557678, 42.03569911, 42.0505648 ]])

In [164]:
indices = indices[0]

In [165]:
closest_movie_ids = movies_and_features.index.values[indices]

In [166]:
closest_movie_ids

array([127206,   8859,   3574,   5740,   3940])

In [167]:
movies[movies["movieId"].isin(closest_movie_ids)][["movieId", "title"]]

,movieId,title
3475,3574,Carnosaur 3: Primal Species (1996)
3836,3940,Slumber Party Massacre III (1990)
5628,5740,Faces of Death: Fact or Fiction? (1999)
8144,8859,SuperBabies: Baby Geniuses 2 (2004)
26791,127206,"People, Places, Things (2015)"
